In [0]:
#hyper parameters used 

lr_init = 0.001
lr_fine_tuned = 0.00001
batch_sz = 64
dropout_rate = 0.3
image_size = 224

In [0]:
import os
import random
import shutil

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications import MobileNet 
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras import layers

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#this assumes the zip file exists on the root location of the Google Drive
#the name "small.zip" indicates the images have been resized to for mobilenet model
!unzip -q "drive/My Drive/small.zip" 

In [0]:
def split_tr_tst(src_dir, tr_dir, tst_dir,tr_split = 0.8):
    items = os.listdir(src_dir)
    #print(items[1])
    random.shuffle(items)
    tst_cnt = round(len(items) * (1-tr_split))
    #print(tst_cnt)
    #print(items[1])
    test_items = items[:tst_cnt]
    train_items = items[tst_cnt:]
    if not(os.path.exists(tst_dir)):
                print('creating ', tst_dir)
                os.makedirs(tst_dir)
    for item in test_items:
        item_path = os.path.join(src_dir,item)
        dest_path = os.path.join(tst_dir, item)
        shutil.copyfile(item_path,dest_path)
    if not(os.path.exists(tr_dir)):
                print('creating ', tr_dir)
                os.makedirs(tr_dir)
    for item in train_items:
        item_path = os.path.join(src_dir,item)
        dest_path = os.path.join(tr_dir, item)
        shutil.copyfile(item_path,dest_path)

In [0]:
#indicate the directory name for the source 
#(source is "./small" if extracted from the default zip file and run by the code above)
#we'll need traning set and testing set

base_path = '.'
src = 'small'
tr = 'train'
tst = 'test'

dir_list = os.listdir(os.path.join(base_path,src))

#count the directories inside train dir for the number of category
#will be used to determine the number of classes of the model
num_categories = len(dir_list)



In [0]:
#only run this if there's still no data split between train and test set
for item in dir_list:
  src_dir = os.path.join(base_path,src,item)
  tr_dir = os.path.join(base_path,tr,item)
  tst_dir = os.path.join(base_path,tst,item)
  
  split_tr_tst(src_dir, tr_dir, tst_dir,tr_split = 0.75)


In [0]:
train_dir = os.path.join(base_path,tr)
test_dir = os.path.join(base_path,tst)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=False,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(image_size, image_size),  
        batch_size=batch_sz,
        class_mode='categorical')

#get dictionary for the label name from the train_generator
class_dict = train_generator.class_indices.items()

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=batch_sz,
        class_mode='categorical')



In [0]:
x_batch, y_batch = next(train_generator)
print(x_batch.shape)

fig = plt.figure(figsize=(8,8))

#show only the first 64 samples if batch size is over 64
for i in range(min(batch_sz,64)):
  fig.add_subplot(8, 8, 1 + i, xticks=[], yticks=[])

  #mobileNet preprocessing convert color range to [-1,1]
  #need to change to [0,1] to show color properly
  plt.imshow((x_batch[i]+1)/2)
plt.show()

In [0]:
x_batch, y_batch = next(test_generator)

print(x_batch.shape)

fig = plt.figure(figsize=(8,8))

#show only the first 64 samples if batch size is over 64
for i in range(min(batch_sz,64)):
  fig.add_subplot(8, 8, 1 + i, xticks=[], yticks=[])
  #mobileNet preprocessing convert color range to [-1,1]
  #need to change to [0,1] to show color properly
  plt.imshow((x_batch[i]+1)/2)
plt.show()

In [0]:
pretrained_model =  MobileNet(input_shape=(image_size,image_size,3), include_top=False)

#freeze all layers
for layer in pretrained_model.layers:
  layer.trainable = False
  
pretrained_model.summary()

In [0]:
def build_model(base_model, num_categories):

  #get the original output (before its actual top layer) from MobileNet
  org_output = base_model.layers[-1].output

  #the other form for building model in functional style
  #since we cant use the other method to modify pre-built models

  #x represent the output of the model we're modifying
  x = GlobalAveragePooling2D()(org_output)
  x = Dense(1024, activation='relu', name='custom_dense1024')(x)
  x = Dropout(dropout_rate)(x)
  x = Dense(num_categories, activation='softmax', name='output')(x)

  #create a new model from the existing pre-trained model stacking with our layers
  return Model(base_model.input, x)


In [0]:
model = build_model(pretrained_model, num_categories)
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=lr_init), 
              metrics=['accuracy'])

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=32,
      epochs=5,
      validation_data=test_generator,
      validation_steps=10,
      verbose=1)

In [0]:
#save weights and rebuilt the model
#the reason we do this is because we need to re-compile the model
#if we want to change the learning rate
#and it causes out-of-memery error 
#if we do that too many times

model.save_weights('model_weights.h5')

#rebuild and re-load weights"
model = build_model(pretrained_model, num_categories)
model.load_weights('model_weights.h5')


#unfreeze the layers down to conv_pw_9

reached_conv_pw_9 = False
for layer in model.layers:
    if layer.name == 'conv_pw_9':
        reached_conv_pw_9 = True
    if reached_conv_pw_9:
        layer.trainable = True
        
model.summary()


In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=lr_fine_tuned), 
              metrics=['accuracy'])

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=32,
      epochs=20,
      validation_data=test_generator,
      validation_steps=10,
      verbose=1)

In [0]:
img_path = 'knife-vol5.png'
img = image.load_img(img_path, target_size=(image_size,image_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

print(model.predict(x))
print(class_dict)


In [0]:
model.save('drive/My Drive/meedthii13_ver2.hdf5')